# Import Package

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

In [ ]:
import torch.utils.data as data
# import io
# import imageio
# from PIL import Image
# from PIL import ImageDraw
# from PIL import ImageFont

from tqdm.notebook import tqdm as tqdm
#from tqdm import tqdm_notebook as tqdm 

import matplotlib.pyplot as plt
import numpy as np

import torchvision.datasets as dset # 可以透過minibatch的方式，去取得訓練的資料
import torchvision.transforms as transforms
import random

from phe import paillier
from multiprocessing import Process, Pool
import os, time
from functools import partial

# Download MNIST Dataset

In [ ]:
# Download dataset
# Load it into dataloader

trans = transforms.Compose([transforms.ToTensor()]) 
train_set = dset.MNIST(root='.', train=True, download=True ,transform=trans)
test_set = dset.MNIST(root='.', train=False,transform=trans)

# Key Generate

In [ ]:
public_key, private_key = paillier.generate_paillier_keypair(n_length=128)

# Network Architecture

In [ ]:
class network(nn.Module):
    def __init__(self):
        super(network,self).__init__()
        self.L1 = nn.Linear(784,128)
        self.L2 = nn.Linear(128,64)
        self.output = nn.Linear(64,10)
    def forward(self , x):
        x = F.relu(self.L1(x))
        x = F.relu(self.L2(x))
        x = self.output(x)  #若是loss用crossentropy 他最後一層會自己用softmax
        return x

# Global Server

In [ ]:
def Encrypt_Multithread(data_array, public_key):
    encrypt_data_array = np.array([public_key.encrypt(float(x)) for x in data_array.reshape(-1)])
    return encrypt_data_array
    #encrypted_num_list.append(encrypt_data_array)
    #print(f"Encrypt thread finishes: {i}")

def Decrypt_Multithread(encrypted_num, private_key):
    #print(f"Decrypt thread: {i}")
    decrypted_data_array = np.array([private_key.decrypt(encrypted_num)])
    return decrypted_data_array
    #decrypted_num_list.append(decrypted_data_array)
    #print(f"Decrypt thread finishes: {i}")


In [ ]:
class GlobalServer():
    def __init__(self):
        def Initial_globalnetwork_dict():
            global_network = network()
            net_dict = global_network.state_dict()

            for key in net_dict.keys():
                net_dict[key] = net_dict[key].numpy()

            return net_dict

        self.global_network_dict = Initial_globalnetwork_dict() #{"key is layer": value is numpy(from tensor to numpy)}
        self.gradient_buffer = dict()
        #self.lr = 0.1
        self.loss_history = []

  
    def AddGradient(self):
        state_dict_tmp = self.global_network_dict
        for key in self.gradient_buffer.keys():
            #print(f"Add layer:{key}")
            #print(key)
            state_dict_tmp[key] += self.gradient_buffer[key]
    
        self.global_network_dict = state_dict_tmp

        self.gradient_buffer.clear()

    def Append_loss(self, loss_sum):
        self.loss_history.append(loss_sum)

    def EncryptParameters(self, public_key):
        params_dict = self.global_network_dict

        pool = Pool(2)
        partial_func = partial(Encrypt_Multithread, public_key=public_key)
        for layer in params_dict.keys():
            shape = params_dict[layer].shape
            params_array = pool.map(partial_func, params_dict[layer].reshape(-1))
            params_concatenate = np.concatenate(params_array)
            params_dict[layer] = params_concatenate.reshape(shape)

        self.global_network_dict = params_dict

# Participant

In [ ]:
from sklearn.metrics import f1_score

class Participant():
    def __init__(self):
        self.local_network = network()
        self.loss_fn = nn.CrossEntropyLoss()
        self.lr = 0.1
        self.optimizer = torch.optim.SGD(params=self.local_network.parameters(), lr = self.lr)
        self.loss_history = []
        self.validate_loss = []
  # ================================================= Download and upload all gradients =================================================== #
    def Download(self, GlobalServer, private_key):
        params_dict = GlobalServer.global_network_dict
        decrypt_params = dict()
        pool = Pool(os.cpu_count())
        partial_func = partial(Decrypt_Multithread, private_key=private_key)
        for key in params_dict.keys():
            shape = params_dict[key].shape
            decrypted_array = pool.map(partial_func, params_dict[key].reshape(-1))
            decrypt_params[key] = torch.Tensor(np.concatenate(decrypted_array).reshape(shape))

        self.local_network.load_state_dict(decrypt_params)

  
    def Upload(self, gradient_dict, GlobalServer, public_key):
        #GlobalServer.gradient_buffer.clear()
        pool = Pool(os.cpu_count())
        #partial_func = partial(Encrypt_Multithread, public_key=public_key)
        for layer in gradient_dict.keys():
            shape = gradient_dict[layer].shape
            gradients_array = gradient_dict[layer].cpu().numpy()
            partial_func = partial(Encrypt_Multithread, public_key=public_key)
            encrypted_array = pool.map(partial_func, gradients_array.reshape(-1))
            encrypted_concatenate = np.concatenate(encrypted_array)
            GlobalServer.gradient_buffer[layer] = encrypted_concatenate.reshape(shape)
            #print(f"Layer:{layer}, GlobalServer:{type(GlobalServer.gradient_buffer[layer])}")
    

    def LocalTraining(self, train_dataset, GlobalServer, public_key, private_key, epochs = 1):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.local_network = self.local_network.cuda()
        tmp_gradient = dict()

        for e in tqdm(range(epochs)):
            epoch_loss_sum = 0
            self.Download(GlobalServer, private_key)
      
            for x , y in tqdm(train_dataset): # tqdm是可以印進度條的package
                ##self.Download(GlobalServer)
                #self.optimizer = torch.optim.SGD(params=self.local_network.parameters(), lr = self.lr)
                #print(GlobalServer.global_network.state_dict())
                #print(self.local_network.state_dict())
                if use_cuda:
                    x = x.cuda()
                    y = y.cuda()
                batch_size = x.shape[0]
                x = x.view(batch_size,-1) # 把data tensor的形狀做改變，-1代表由pytorch決定要變多少
                net_out = self.local_network(x) # 把x丟進net去train，得到output
                loss = self.loss_fn(net_out , y) # 把train出來的結果和ground truth去算loss
                epoch_loss_sum += float(loss.item())
                 # 做backpropagation
                self.optimizer.zero_grad() #先把optimizer清空
                loss.backward()
                #self.local_network.weight.grad , net.L3.bias.gradnet.L3.weight.grad , net.L3.bias.grad
                self.optimizer.step() # 把算完的gradient套在network的parameter
                for layer, param in self.local_network.named_parameters():
                    if (tmp_gradient.get(layer) == None): 
                        tmp_gradient[layer] = -1 * param.grad * self.lr
                    else:
                        tmp_gradient[layer] -= param.grad * self.lr
      
            #Encrypted_grdd = PaillierEncrypt(tmp_gradient)  
            self.Upload(tmp_gradient, GlobalServer, public_key)


            GlobalServer.AddGradient()
            tmp_gradient.clear()
            self.loss_history.append(epoch_loss_sum)
            Server.Append_loss(epoch_loss_sum)

    def LocalTesting(self, test_dataset):
        correct_count = 0
        total_testdata = 0
        epoch_loss_sum = 0
        use_cuda = torch.cuda.is_available()
        Y = []
        Output = []

        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.local_network = self.local_network.cuda()

        for x,y in test_dataset:
            if use_cuda:
                x = x.cuda()
                y = y.cuda()
            batch_size = x.shape[0]
            total_testdata += batch_size
            x = x.view(batch_size , -1)
            output = self.local_network(x).max(1)[1]
            net_out = self.local_network(x) 

            # validation loss
            loss = self.loss_fn(net_out , y)
            epoch_loss_sum += float(loss.item())

            Y = Y + y.tolist()
            Output = Output + output.tolist()
            correct_count += torch.sum(output==y).item()
        correct_count = correct_count
        #print("Y:", Y)
        #print("Output:", Output)
        self.validate_loss.append(epoch_loss_sum)
        f1 = f1_score(Y, Output, average = "macro")
        print('accuracy rate',correct_count/total_testdata)
        print("f1-score:", f1)

        return correct_count/total_testdata

  # ================================================== 分解 Download and (training, upload) Download另外叫 ============================================ #
    def Training(self, train_dataset, GlobalServer, public_key, epochs = 1): # 分解Download and Upload
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.local_network = self.local_network.cuda()

        for e in tqdm(range(epochs)):
            epoch_loss_sum = 0
            tmp_gradient = dict()
            for x , y in tqdm(train_dataset): # tqdm是可以印進度條的package
                #self.Download(GlobalServer)
                #self.optimizer = torch.optim.SGD(params=self.local_network.parameters(), lr = self.lr)
                #print(GlobalServer.global_network.state_dict())
                #print(self.local_network.state_dict())
            if use_cuda:
                x = x.cuda()
                y = y.cuda()
            batch_size = x.shape[0]
            x = x.view(batch_size,-1) # 把data tensor的形狀做改變，-1代表由pytorch決定要變多少
            net_out = self.local_network(x) # 把x丟進net去train，得到output
            loss = self.loss_fn(net_out , y) # 把train出來的結果和ground truth去算loss
            epoch_loss_sum += float(loss.item())
             # 做backpropagation
            self.optimizer.zero_grad() #先把optimizer清空
            loss.backward()
            #self.local_network.weight.grad , net.L3.bias.gradnet.L3.weight.grad , net.L3.bias.grad
            self.optimizer.step() # 把算完的gradient套在network的parameter
            for layer, param in self.local_network.named_parameters():
                if (tmp_gradient.get(layer) == None): 
                    tmp_gradient[layer] = -1 * param.grad * self.lr
                else:
                    tmp_gradient[layer] -= param.grad * self.lr     
      
            self.Upload(tmp_gradient, GlobalServer, public_key)
        
            GlobalServer.AddGradient()
            tmp_gradient.clear()
            self.loss_history.append(epoch_loss_sum)

  # ======================================================================== 只有training, Download upload 另外呼叫 ======================================== #
    def OnlyTraining(self, train_dataset, GlobalServer, epochs = 1): 
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.local_network = self.local_network.cuda()

        for e in tqdm(range(epochs)):
            epoch_loss_sum = 0

            for x , y in tqdm(train_dataset): # tqdm是可以印進度條的package
                #self.Download(GlobalServer)
                #self.optimizer = torch.optim.SGD(params=self.local_network.parameters(), lr = self.lr)
                #print(GlobalServer.global_network.state_dict())
                #print(self.local_network.state_dict())
                if use_cuda:
                    x = x.cuda()
                    y = y.cuda()
                batch_size = x.shape[0]
                x = x.view(batch_size,-1) # 把data tensor的形狀做改變，-1代表由pytorch決定要變多少
                net_out = self.local_network(x) # 把x丟進net去train，得到output
                loss = self.loss_fn(net_out , y) # 把train出來的結果和ground truth去算loss
                epoch_loss_sum += float(loss.item())
                 # 做backpropagation
                self.optimizer.zero_grad() #先把optimizer清空
                loss.backward()
                #self.local_network.weight.grad , net.L3.bias.gradnet.L3.weight.grad , net.L3.bias.grad
                self.optimizer.step() # 把算完的gradient套在network的parameter
        
      
            self.loss_history.append(epoch_loss_sum)
  
  # 直接上傳參數
    def UploadParameters(self, GlobalServer):
        GlobalServer.global_network.load_state_dict(self.local_network.state_dict())

  # 直接下載參數 透過layerlist選擇要下載哪幾層的
    def DownloadLayer(self, Server, layer_list):
        #network_dict = dict()
        local_network_dict = self.local_network.state_dict()
        server_netwrok_dict = Server.global_network.state_dict()

        for layer in layer_list:
            local_network_dict[layer] = server_netwrok_dict[layer]
    
        self.local_network.load_state_dict(local_network_dict)
  
  # 上傳gradient 透過layerlist選擇要上傳哪幾層
    def UploadLayer(self, gradient_dict, GlobalServer, layer_list):
        GlobalServer.gradient_buffer.clear()
    
        for layer in layer_list:
            GlobalServer.gradient_buffer[layer] = -1 * gradient_dict[layer] * self.lr
    
        #GlobalServer.lr = self.lr
        #print(GlobalServer.gradient_buffer)
  
  # Training 透過layer_list看要上傳哪幾層的gradient Download要在前面呼叫
    def TrainingUploadLayer(self, train_dataset, GlobalServer, layer_list, epochs = 1):
        use_cuda = torch.cuda.is_available()
        if use_cuda:
            self.local_network = self.local_network.cuda()

        for e in tqdm(range(epochs)):
            epoch_loss_sum = 0
            tmp_gradient = dict()
            for x , y in tqdm(train_dataset): # tqdm是可以印進度條的package
                #self.Download(GlobalServer)
                #self.optimizer = torch.optim.SGD(params=self.local_network.parameters(), lr = self.lr)
                #print(GlobalServer.global_network.state_dict())
                #print(self.local_network.state_dict())
                if use_cuda:
                    x = x.cuda()
                    y = y.cuda()
                batch_size = x.shape[0]
                x = x.view(batch_size,-1) # 把data tensor的形狀做改變
                net_out = self.local_network(x) # 把x丟進net去train，得到output
                loss = self.loss_fn(net_out , y) # 把train出來的結果和ground truth去算loss
                epoch_loss_sum += float(loss.item())
                 # 做backpropagation
                self.optimizer.zero_grad() #先把optimizer清空
                loss.backward()
                #self.local_network.weight.grad , net.L3.bias.gradnet.L3.weight.grad , net.L3.bias.grad
                self.optimizer.step() # 把算完的gradient套在network的parameter
                for layer, param in zip(self.local_network.state_dict().keys(), self.local_network.parameters()):
                    if (tmp_gradient.get(layer) == None): 
                        tmp_gradient[layer] = -1 * param.grad
                    else:
                        tmp_gradient[layer] -= param.grad

            self.UploadLayer(tmp_gradient, GlobalServer, layer_list)
        
            GlobalServer.AddGradient()
            tmp_gradient.clear()
            self.loss_history.append(epoch_loss_sum)
  
  # 直接上傳某一層參數
    def UploadParametersLayer(self, GlobalServer, layer_list):
        global_state_dict = GlobalServer.global_network.state_dict()
        local_state_dict = self.local_network.state_dict()
        for layer in layer_list:
            global_state_dict[layer] = local_state_dict[layer]
    
        GlobalServer.global_network.load_state_dict(global_state_dict)


# Generate Participant and Assign Dataset To Them

In [ ]:
def GenerateParticipant(num = 1):
    P_dict = dict()
    for i in range(num):
        s = 'P'
        s = s + str(i)
        P_dict[s] = Participant()

    return P_dict

def SplitData(train_set, test_set, num):
    train_split = int(len(train_set) / num)
    test_split = int(len(test_set) / num)

    portions = [train_split] * num
    TrainSet_list = [None] * num
    TrainSet_list = data.random_split(train_set, portions)

    portions = [test_split] * num
    TestSet_list = [None] * num
    TestSet_list = data.random_split(test_set, portions)

    TrainDataSet_dict = dict()
    TestDataSet_dict = dict()

    # mini batch
    for i in range(num):
        s = 'P'
        s = s + str(i)
        TrainDataSet_dict[s] = data.DataLoader(dataset =  TrainSet_list[i], batch_size=50, shuffle=True)
        TestDataSet_dict[s] = data.DataLoader(dataset =  TestSet_list[i], batch_size=50, shuffle=True)

    return TrainDataSet_dict, TestDataSet_dict


# DSSGD
Reference: https://www.cs.cornell.edu/~shmat/shmat_ccs15.pdf, https://ieeexplore.ieee.org/document/8241854

In [ ]:
#測試準確率的 test dataset (全部10,000的資料)
test_dataset = data.DataLoader(dataset =  test_set, batch_size=50,shuffle=True)

## Round Robin

In [ ]:
# 用來檢測accuracy
test_P = GenerateParticipant(1) 

# Dataset and Participant Gernerate
n = 10
Participant_dict = GenerateParticipant(n)
TrainData_dict, TestData_dict = SplitData(train_set, test_set, n)

Server = GlobalServer()
Server.EncryptParameters(public_key)

In [ ]:
global_accuracy_list = []
global_loss_history = []
rounds = 1 # round robin要做幾輪

for i in tqdm(range(rounds)):
    for j in range(n):
        s = 'P'
        s = s + str(j)
        #Participant_dict[s].Download(Server, private_key)
        #Participant_dict[s].Training(TrainData_dict[s], Server, public_key, 1)
        Participant_dict[s].LocalTraining(TrainData_dict[s], Server, public_key, private_key, 1)
        test_P["P0"].Download(Server, private_key)
        accu = test_P["P0"].LocalTesting(test_dataset)
        global_accuracy_list.append(accu)
        global_loss_history.append(Participant_dict[s].loss_history[i])

### 怕google colab當掉 所以要測10次取平均的時候一次一次做

In [ ]:
# 只有第一次要generate 剩下不要run 不然會蓋掉前面資料
global_accuracy_array = np.empty((10, 100))
global_loss_array = np.empty((10, 100))

In [ ]:
global_accuracy_array[0][:] = np.array(global_accuracy_list)

In [ ]:
global_loss_array[0][:] = np.array(global_loss_history)

## Asychronous

In [ ]:
cnt = 0
up_cnt = 0

k = 1 # 上一個participant訓練完後 接下來要幾(k)個participan去訓練並上傳
rounds = 1 # 要做幾輪
m_list = list(range(9, 10)) # 不同的m，因為怕google colab當掉，所以建議一個一個去試 尤其是epoch = 10 epcoh = 1還好

epoch_set = [1]#因為怕當掉所以分別去試 [1, 10] # 每個participant經過的epoch，Server經過幾個epoch要再乘上有幾個participant

asy_accuracy_dict = dict() # key是代表epoch為多少 value為list index由0~9分別為m = 1~10


up_cnt = 0 # 第幾個participant
#accu_list = []
accu_sum = 0
average_accu = 0

times = 1 #不同的m要做幾次取平均，因為怕colab掛掉，所以這邊設1，每次把數據紀錄起來

for m in m_list:
    for _ in range(times):
        n = 10
        Participant_dict = GenerateParticipant(n)
        TrainData_dict, TestData_dict = SplitData(train_set, test_set, n)

        Server = GlobalServer()
        Server.EncryptParameters(public_key)

        #cnt = 0
        #up_cnt = 0
        Participant_book = list(Participant_dict.keys())
        P = GenerateParticipant(1)
        for _ in range(rounds):
            cnt = 0
            up_cnt = 0
            Upload_list = random.sample(Participant_book, n)
            for key in Upload_list:
                #print(key)
                for i in range(m):
                    if (cnt == n):
                        break
                    #print(cnt)
                    Participant_dict[Upload_list[cnt]].Download(Server, private_key)
                    cnt += 1

                for _ in range(k):
                    if (up_cnt == n):
                        break 
                    Participant_dict[Upload_list[up_cnt]].Training(TrainData_dict[Upload_list[up_cnt]], Server, public_key, 1)
                    up_cnt += 1
            #print(round)
        P["P0"].Download(Server, private_key)
        accu_sum += P["P0"].LocalTesting(test_dataset)
        #accu_list.append(P["P0"].LocalTesting(test_dataset))
        #print(cnt)
        #accu_list.clear()
    average_accu = accu_sum / times #除上做幾次
    if (asy_accuracy_dict.get('m'+str(m)+'totalepochs:'+str(r * n)) == None):
        asy_accuracy_dict['m'+str(m)+'totalepochs:'+str(r * n)] = []

    asy_accuracy_dict['m'+str(m)+'totalepochs:'+str(r * n)].append(average_accu)
    accu_sum = 0
  

# Upload Parameters

## Random Order

In [ ]:
# Initial
n = 10
Participant_dict = GenerateParticipant(n)
TrainData_dict, TestData_dict = SplitData(train_set, test_set, n)

Server = GlobalServer()
Server.EncryptParameters(public_key)

Participant_book = list(Participant_dict.keys())
Upload_list = random.sample(Participant_book, n)

# 驗證準確性
test_P = GenerateParticipant()

In [ ]:
global_accuracy_list = []
loss_history = []
rounds = 1

for i in tqdm(range(rounds)):
    #Upload_list = random.sample(Participant_book, n)
    for key in Upload_list:
        Participant_dict[key].Download(Server)
        Participant_dict[key].OnlyTraining(TrainData_dict[key], Server, 1)
        Participant_dict[key].UploadParameters(Server)
        test_P["P0"].Download(Server)
        accu = test_P["P0"].LocalTesting(test_dataset)
        global_accuracy_list.append(accu)
        loss_history.append(Participant_dict[key].loss_history[i])

## Asychronous

In [ ]:
k = 1 # 上一部分participants download完後 接下來要幾個participants去訓練並上傳
rounds = 1 # 要做幾輪
m_list = list(range(10, 11)) # 不同的m，因為我用google colab, 怕google colab當掉，所以我是一個一個去試 

#epoch_set = [1] # 每個participant做完1 epoch上傳奇gradients

asy_accuracy_dict = dict() # key是代表Server經過的總epochs為多少 value為分別為m = 1~10的accuracy
#asy_accuracy_dict["m4totalepochs:100"] = []
#asy_accuracy_dict["e10"] = []


#accu_list = []
accu_sum = 0
average_accu = 0

times = 1 #要做幾次去測試
epochs = 1 # 每個participants做幾個epoch後上傳

for m in m_list:
    for _ in range(times):
        n = 10
        Participant_dict = GenerateParticipant(n)
        TrainData_dict, TestData_dict = SplitData(train_set, test_set, n)
        Server = GlobalServer()
        Server.EncryptParameters(public_key)
        up_cnt = 0
        cnt = 0
        Participant_book = list(Participant_dict.keys())
        P = GenerateParticipant(1) # 檢測global server的accuracy
        for _ in range(rounds):
            cnt = 0
            up_cnt = 0
            Upload_list = random.sample(Participant_book, n)

            for key in Upload_list:
                Participant_dict[key].Download(Server)
                Participant_dict[key].OnlyTraining(TrainData_dict[key], Server, epochs)
                cnt += 1
                if cnt % m == 0:
                    Participant_dict[Upload_list[up_cnt]].UploadParameters(Server)
                up_cnt += 1
            print(up_cnt)   
      
            for _ in range(n - up_cnt):
                Participant_dict[Upload_list[up_cnt]].UploadParameters(Server)
                #Server.AddGradient()
                up_cnt += 1

            P["P0"].Download(Server)
            accu_sum += P["P0"].LocalTesting(test_dataset)
      
    average_accu = accu_sum / times #除上做幾次
    if (asy_accuracy_dict.get('m'+str(m)+'totalepochs:'+str(r * n * epochs)) == None):
        asy_accuracy_dict['m'+str(m)+'totalepochs:'+str(r * n * epochs)] = []

    asy_accuracy_dict['m'+str(m)+'totalepochs:'+str(r * n * epochs)].append(average_accu)
    accu_sum = 0